# Custom Word Embeddings

In [0]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [0]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

In [0]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

## Let's first find the total number of words in our corpus:

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk.tokenize import word_tokenize

all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

In [16]:
unique_words = set(all_words)
print(len(unique_words))

45


### In the output you will see "45", which is the number of unique words in our corpus. We will add a buffer of 5 to our vocabulary size and will set the value of vocab_length to 50.

The Embedding layer expects the words to be in numeric form. Therefore, we need to convert the sentences in our corpus to numbers. One way to convert text to numbers is by using the one_hot function from the keras.preprocessing.text library. The function takes sentence and the total length of the vocabulary and returns the sentence in numeric form.

In [0]:
vocab_length = 50

In [20]:
for sent in corpus:
  print(sent, one_hot(sent, vocab_length))

This is an excellent movie [36, 44, 29, 27, 39]
The move was fantastic I like it [29, 37, 5, 22, 8, 28, 42]
You should watch it is brilliant [3, 11, 14, 42, 44, 11]
Exceptionally good [27, 42]
Wonderfully directed and executed I like it [37, 22, 43, 25, 8, 28, 42]
Its a fantastic series [23, 39, 22, 49]
Never watched such a brillent movie [9, 44, 37, 39, 36, 39]
It is a Wonderful movie [42, 44, 39, 8, 39]
horrible acting [45, 46]
waste of money [35, 1, 29]
pathetic picture [33, 22]
It was very boring [42, 5, 25, 24]
I did not like the movie [8, 23, 19, 28, 29, 39]
The movie was horrible [29, 39, 5, 45]
I will not recommend [8, 39, 19, 42]
The acting is pathetic [29, 46, 44, 33]


In [21]:
embedded_sentences = [one_hot(sent, vocab_length) for sent in corpus]
print(embedded_sentences )

[[36, 44, 29, 27, 39], [29, 37, 5, 22, 8, 28, 42], [3, 11, 14, 42, 44, 11], [27, 42], [37, 22, 43, 25, 8, 28, 42], [23, 39, 22, 49], [9, 44, 37, 39, 36, 39], [42, 44, 39, 8, 39], [45, 46], [35, 1, 29], [33, 22], [42, 5, 25, 24], [8, 23, 19, 28, 29, 39], [29, 39, 5, 45], [8, 39, 19, 42], [29, 46, 44, 33]]


In [0]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

In [25]:
length_long_sentence

7

In [26]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[36 44 29 27 39  0  0]
 [29 37  5 22  8 28 42]
 [ 3 11 14 42 44 11  0]
 [27 42  0  0  0  0  0]
 [37 22 43 25  8 28 42]
 [23 39 22 49  0  0  0]
 [ 9 44 37 39 36 39  0]
 [42 44 39  8 39  0  0]
 [45 46  0  0  0  0  0]
 [35  1 29  0  0  0  0]
 [33 22  0  0  0  0  0]
 [42  5 25 24  0  0  0]
 [ 8 23 19 28 29 39  0]
 [29 39  5 45  0  0  0]
 [ 8 39 19 42  0  0  0]
 [29 46 44 33  0  0  0]]


In [27]:
model = Sequential()
model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [28]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 7, 20)             1000      
_________________________________________________________________
flatten_1 (Flatten)          (None, 140)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 141       
Total params: 1,141
Trainable params: 1,141
Non-trainable params: 0
_________________________________________________________________
None


You can see that the first layer has 1000 trainable parameters. This is because our vocabulary size is 50 and each word will be presented as a 20 dimensional vector. Hence the total number of trainable parameters will be 1000. Similarly, the output from the embedding layer will be a sentence with 7 words where each word is represented by a 20 dimensional vector. However, when the 2D output is flattened, we get a 140 dimensional vector (7 x 20). The flattened vector is directly connected to the dense layer that contains 1 neuran.

In [29]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)




Epoch 1/100





16/16 [==============================] - 1s 46ms/step - loss: 0.6856 - acc: 0.5000
Epoch 2/100
16/16 [==============================] - 0s 167us/step - loss: 0.6830 - acc: 0.5625
Epoch 3/100
16/16 [==============================] - 0s 142us/step - loss: 0.6800 - acc: 0.6250
Epoch 4/100
16/16 [==============================] - 0s 209us/step - loss: 0.6769 - acc: 0.6250
Epoch 5/100
16/16 [==============================] - 0s 157us/step - loss: 0.6734 - acc: 0.6250
Epoch 6/100
16/16 [==============================] - 0s 307us/step - loss: 0.6699 - acc: 0.6250
Epoch 7/100
16/16 [==============================] - 0s 212us/step - loss: 0.6665 - acc: 0.6875
Epoch 8/100
16/16 [==============================] - 0s 183us/step - loss: 0.6631 - acc: 0.7500
Epoch 9/100
16/16 [==============================] - 0s 178us/step - loss: 0.6597 - acc: 0.7500
Epoch 10/100
16/16 [==============================] - 0s 138us/step - loss: 0.6562 - acc: 0.8750
Epoch 11/100
16/16 [============

In [30]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 93.750000
